## Kaggle Data Science Survey

### Import libraries

In [1]:
import pandas as pd
import urllib.request
import zipfile

### Download and extract data

In [2]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
'kaggle-survey-2018.zip' # URL to download the data

In [3]:
fname = 'kaggle-survery-2018.zip' # save file as this name
member_name = 'multipleChoiceResponses.csv' # name of the csv file

In [4]:
# Extract the csv file from the zip file
def extract_zip(src, dst, member_name): 
    """
    Extract a member file from a zip file and read it into a Pandas DataFrame.
    
    Parameters
    ----------    
    src (str): path to the zip file
    dst (str): path to the destination file
    member_name (str): name of the member file to be read into a DataFrame
    
    Returns
    ----------
    pandas.DataFrame: DataFrame containing the member file
    """
    url = src # URL to download the data
    fname = dst # save file as this name
    fin = urllib.request.urlopen(url) # open the url
    data = fin.read() # read the data

    with open(dst, mode='wb') as fout: # save the data to a file
        fout.write(data) # write the data to the file
    with zipfile.ZipFile(dst) as z: # open the zip file
        kag = pd.read_csv(z.open(member_name)) # read the csv file
        kag_questions = kag.iloc[0] # get the questions
        raw = kag.iloc[1:] # get the answers
        return raw # return the answers

In [5]:
raw = extract_zip(url, fname, member_name) # extract the data

/var/folders/02/t1w3xsh916z60p5jhmd5xbc00000gn/T/ipykernel_41031/854491727.py:22: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(z.open(member_name)) # read the csv file


### Build a cleanup pipeline

In [10]:
# Build a cleanup function that uses Sckit-Learn pipelines
def tweak_kag(df_: pd.DataFrame) -> pd.DataFrame:
    """
    Tweak the Kaggle survey data and return a new DataFrame.
    This function takes a Pandas DataFrame containing Kaggle
    survey data as input and returns a new DataFrame. The
    modifications include extracting and transforming certain
    columns, renaming columns, and selecting a subset of columns.
    
    Parameters
    ----------
    df_ : pd.DataFrame
        The input DataFrame containing Kaggle survey data.
    Returns
    -------
    pd.DataFrame
        The output DataFrame containing the transformed data.
    """

    return (df_
            .assign(age=df_.Q2.str.slice(0, 2).astype(int),
                    education=df_.Q4.replace({'Master’s degree': 18, 
                                              'Bachelor’s degree': 16,
                                              'Doctoral degree': 20,
            'Some college/university study without earning a bachelor’s degree': 13,
                                              'Professional degree': 19,
                                              'I prefer not to answer': None,
                                              'No formal education past high school': 12}), 
                    major=(df_.Q5
                                .pipe(topn, n=3)
                                .replace({
                                            'Computer science (software engineering, etc.)': 'cs',
                                            'Engineering (non-computer focused)': 'eng',
                                            'Mathematics or statistics': 'stat'})
                                ),
                                years_exp=(df_.Q8.str.replace('+','', regex=False)
                                    .str.split('-', expand=True) .iloc[:,0]
                                    .astype(float)),
                                compensation=(df_.Q9.str.replace('+','', regex=False) 
                                              .str.replace(',','', regex=False) .str.replace('500000', '500', regex=False)
                        .str.replace('I do not wish to disclose my approximate yearly compensation', '0', regex=False)
                                    .str.split('-', expand=True) .iloc[:,0]
                                    .fillna(0)
                                    .astype(int)
                                    .mul(1_000)
                                    ),
                                    python=df_.Q16_Part_1.fillna(0).replace('Python', 1),
                                    r=df_.Q16_Part_2.fillna(0).replace('R', 1),
                                    sql=df_.Q16_Part_3.fillna(0).replace('SQL', 1)
                                )#assign
                            .rename(columns=lambda col:col.replace(' ', '_'))
                            .loc[:, 'Q1,Q3,age,education,major,years_exp,compensation,'
                                    'python,r,sql'.split(',')]
                        )              

In [11]:
def topn(ser, n=5, default='other'): 
    """
    Replace all values in a Pandas Series that are not among
    the top `n` most frequent values with a default value.
    This function takes a Pandas Series and returns a new
    Series with the values replaced as described above. The
    top `n` most frequent values are determined using the
    `value_counts` method of the input Series.
    
    Parameters
    ----------
    ser : pd.Series
        The input Series.
    n : int, optional
        The number of most frequent values to keep. The
        default value is 5.
    default : str, optional
        The default value to use for values that are not among
        the top `n` most frequent values. The default value is
        'other'.
    
    Returns
    -------
    pd.Series
        The modified Series with the values replaced.
    """ 
    counts = ser.value_counts() # get the value counts
    return ser.where(ser.isin(counts.index[:n]), default) # return the modified series

Create the TweakKagTransformer class to wrap tweak_kag and embed cleanup logic into the pipeline functionality of Scikit-Learn.

In [13]:
from feature_engine import encoding, imputation
from sklearn import base, pipeline

In [14]:
class TweakKagTransformer(base.BaseEstimator, base.TransformerMixin):
    """
        A transformer for tweaking Kaggle survey data.
        This transformer takes a Pandas DataFrame containing
        Kaggle survey data as input and returns a new version of
        the DataFrame. The modifications include extracting and
        transforming certain columns, renaming columns, and
        selecting a subset of columns.
    Parameters
    ----------
        ycol : str, optional
        The name of the column to be used as the target variable.
        If not specified, the target variable will not be set.
    Attributes
    ----------
        ycol : str
    """
    def __init__(self, ycol=None):
        self.ycol = ycol

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return tweak_kag(X)

Create a new function, get_rawX_y, which takes the original data and return an X DataFrame and a y Series ready to feed into the pipeline for further cleanup.

In [15]:
def get_rawX_y(df, y_col):
    raw = (df.query('Q3.isin(["United States of America", "China", "India"]) '
                    'and Q6.isin(["Data Scientist", "Software Engineer"])') 
    )
    return raw.drop(columns=[y_col]), raw[y_col]

Create the pipeline and prepare to run the code

In [17]:
# Create a pipeline to preprocess the data
kag_pl = pipeline.Pipeline(
    [('tweak', TweakKagTransformer()),
     ('cat', encoding.OneHotEncoder(top_categories=5, drop_last=True,
                                    variables=['Q1', 'Q3', 'major'])),
     ('num_impute', imputation.MeanMedianImputer(imputation_method='median',
                                               variables=['education', 'years_exp']))
    ]
)

### Split the data

In [19]:
from sklearn import model_selection
kag_X, kag_y = get_rawX_y(raw, 'Q6')
kag_X_train, kag_X_test, kag_y_train, kag_y_test = model_selection.train_test_split(kag_X, kag_y, test_size=.3, stratify=kag_y, random_state=42)

### Fit the model

In [20]:
X_train = kag_pl.fit_transform(kag_X_train, kag_y_train)
X_test = kag_pl.transform(kag_X_test)
print(X_train)

       age  education  years_exp  compensation  python  r  sql  Q1_Male  \
587     25       18.0        4.0             0       1  0    1        1   
3065    22       16.0        1.0         10000       1  0    0        1   
8435    22       18.0        1.0             0       1  0    0        1   
3110    40       20.0        3.0        125000       1  0    1        0   
16372   45       12.0        5.0        100000       1  0    1        1   
...    ...        ...        ...           ...     ... ..  ...      ...   
16608   25       16.0        2.0         10000       0  0    1        1   
7325    18       16.0        1.0         30000       1  0    1        1   
21810   18       16.0        2.0             0       0  0    0        1   
4917    25       18.0        1.0        100000       1  0    1        1   
639     25       18.0        1.0         10000       1  0    0        0   

       Q1_Female  Q1_Prefer not to say  Q1_Prefer to self-describe  \
587            0             

### Check the labels

In [21]:
kag_y_train

587      Software Engineer
3065        Data Scientist
8435        Data Scientist
3110        Data Scientist
16372    Software Engineer
               ...        
16608    Software Engineer
7325     Software Engineer
21810       Data Scientist
4917        Data Scientist
639         Data Scientist
Name: Q6, Length: 2110, dtype: object